In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from IPython.display import display, Markdown

In [ ]:
def prd(l):
    p = 1
    for i in l:
        p *= int(i)
    return p

def layerStrs(l):
    c = l.get_config()
    #print(l.__class__.__name__, c)
    name = c.get('name', '')
    unit = l.__class__.__name__
    act = c.get('activation', '')
    size = c.get('kernel_size', '')
    if not size:
        size = c.get('pool_size', '')
    if not size:
        size = c.get('rate', '')
    iShape = l.input_shape[1:]
    oShape = l.output_shape[1:]
    nParam = sum( [ prd(w.shape) for w in l.weights ] )
    nData = prd(oShape)
    return [name, unit, act, size, iShape, oShape, nParam, nData] 

def prn(self):
    strs = ['|Layer|Name|Unit|Activation|Size|iShape|oShape|nParam|nData|',
            '|-----|-----|----|----|------|------|------|-----|']
    tp = 0
    for i,l in enumerate(self.layers):
        f = tuple([i+1,]+layerStrs(l))
        strs.append('|%s|%s|%s|%s|%s|%s|%s|%s|%s|' % f )
        tp += f[7]
    t = '\n'.join(strs)
    display(Markdown(t))
    print(tp)


In [1]:
batch_size = 128
num_classes = 10
epochs = 60

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(27, kernel_size=(7, 7), input_shape=input_shape, use_bias=0,))
model.add(Dropout(.05))
model.add(MaxPooling2D(pool_size=(9, 9), strides=4))
model.add(Conv2D(14, kernel_size=(1, 1), activation='relu', use_bias=0)) 
model.add(Flatten())
model.add(Dropout(.05))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

prn(model) # just print a table of the model + param coubnt.


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


/Users/michaelnaunton/anaconda/envs/py36-test/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/michaelnaunton/anaconda/envs/py36-test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/michaelnaunton/anaconda/envs/py36-test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


|Layer|Name|Unit|Activation|Size|iShape|oShape|nParam|nData|
|-----|-----|----|----|------|------|------|-----|
|1|conv2d_1|Conv2D|linear|(7, 7)|(28, 28, 1)|(22, 22, 27)|1323|13068|
|2|dropout_1|Dropout||0.05|(22, 22, 27)|(22, 22, 27)|0|13068|
|3|max_pooling2d_1|MaxPooling2D||(9, 9)|(22, 22, 27)|(4, 4, 27)|0|432|
|4|conv2d_2|Conv2D|relu|(1, 1)|(4, 4, 27)|(4, 4, 14)|378|224|
|5|flatten_1|Flatten|||(4, 4, 14)|(224,)|0|224|
|6|dropout_2|Dropout||0.05|(224,)|(224,)|0|224|
|7|dense_1|Dense|softmax||(224,)|(10,)|2250|10|

3951
Train on 60000 samples, validate on 10000 samples
Epoch 1/60
60000/60000 [==============================] - 49s 824us/step - loss: 0.3863 - acc: 0.8865 - val_loss: 0.1086 - val_acc: 0.9677
Epoch 2/60
60000/60000 [==============================] - 51s 848us/step - loss: 0.1231 - acc: 0.9625 - val_loss: 0.0920 - val_acc: 0.9712
Epoch 3/60
60000/60000 [==============================] - 44s 731us/step - loss: 0.0974 - acc: 0.9703 - val_loss: 0.0638 - val_acc: 0.9792
Epoch 4/60
60000/60000 [==============================] - 39s 654us/step - loss: 0.0824 - acc: 0.9747 - val_loss: 0.0572 - val_acc: 0.9824
Epoch 5/60
60000/60000 [==============================] - 38s 641us/step - loss: 0.0743 - acc: 0.9769 - val_loss: 0.0515 - val_acc: 0.9843
Epoch 6/60
60000/60000 [==============================] - 42s 707us/step - loss: 0.0681 - acc: 0.9792 - val_loss: 0.0498 - val_acc: 0.9846
Epoch 7/60
60000/60000 [==============================] - 45s 753us/step - loss: 0.0645 - acc: 0.9805 - val_los

60000/60000 [==============================] - 54s 894us/step - loss: 0.0237 - acc: 0.9919 - val_loss: 0.0314 - val_acc: 0.9918
Epoch 60/60
60000/60000 [==============================] - 50s 827us/step - loss: 0.0236 - acc: 0.9925 - val_loss: 0.0308 - val_acc: 0.9917
Test loss: 0.0307828644287074
Test accuracy: 0.9917
